In [4]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df["Target"] = housing.target

In [6]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df[housing.feature_names], df["Target"], test_size=0.2, random_state=42
)

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [10]:
input_size = X_train.shape[1]   # 8 features
output_size = y_train.shape[1]  # 1 target
model = nn.Linear(input_size, output_size)

In [16]:
learning_rate = 0.01
num_epochs = 2500
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [17]:
for epoch in range(num_epochs):
    model.train()
    y_pred = model(X_train)
    l  = loss(y_pred, y_train)
    l.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1) % 10 == 0:
        [w, b] = model.parameters()
        print(f"Epoch {epoch+1}, Loss: {l.item():.4f}, Weights: {w.detach().numpy()}, Bias: {b.item():.4f}")

Epoch 10, Loss: 0.5503, Weights: [[ 0.8805428   0.18376845 -0.24044426  0.24483253  0.01839063 -0.04705966
  -0.40794656 -0.37889645]], Bias: 2.0719
Epoch 20, Loss: 0.5494, Weights: [[ 0.88096905  0.18295005 -0.24285541  0.24762188  0.01811654 -0.04700026
  -0.4139779  -0.3850603 ]], Bias: 2.0719
Epoch 30, Loss: 0.5485, Weights: [[ 0.8813794   0.18214186 -0.24521287  0.25035378  0.01784506 -0.04694076
  -0.4199289  -0.39114064]], Bias: 2.0719
Epoch 40, Loss: 0.5477, Weights: [[ 0.88177437  0.18134394 -0.24751759  0.2530293   0.01757631 -0.04688127
  -0.42580077 -0.3971386 ]], Bias: 2.0719
Epoch 50, Loss: 0.5469, Weights: [[ 0.88215446  0.18055633 -0.24977055  0.25564942  0.01731039 -0.04682188
  -0.43159464 -0.4030554 ]], Bias: 2.0719
Epoch 60, Loss: 0.5461, Weights: [[ 0.88252     0.179779   -0.2519727   0.25821513  0.01704739 -0.04676267
  -0.4373115  -0.40889236]], Bias: 2.0719
Epoch 70, Loss: 0.5453, Weights: [[ 0.8828714   0.17901193 -0.25412503  0.26072744  0.01678738 -0.04670371

In [18]:
with torch.no_grad():
    y_pred_test = model(X_test)
    test_loss = loss(y_test, y_pred_test)
    print(f"\nTest Loss (MSE): {test_loss.item():.5f}")



Test Loss (MSE): 0.55681
